In [12]:
import pynq
import asyncio
overlay = pynq.Overlay('design_1_wrapper.bit')

In [13]:
#overlay?

In [14]:
timer = overlay.axi_timer_0
timer._interrupts

{'interrupt': {'controller': 'axi_intc_0',
  'index': 0,
  'fullpath': 'axi_timer_0/interrupt'}}

In [15]:
timer.register_map

RegisterMap {
  TCSR0 = Register(MDT0=0, UDT0=0, GENT0=0, CAPT0=0, ARHT0=0, LOAD0=0, ENIT0=0, ENT0=0, T0INT=0, PWMA0=0, ENALL=0, CASC=0),
  TLR0 = Register(TCLR0=0),
  TCR0 = Register(TCR0=0),
  TCSR1 = Register(MDT1=0, UDT1=0, GENT1=0, CAPT1=0, ARHT1=0, LOAD1=0, ENIT1=0, ENT1=0, T1INT=0, PWMA1=0, ENALL=0),
  TLR1 = Register(TCLR1=0),
  TCR1 = Register(TCR1=0)
}

In [16]:
interrupt = timer.interrupt

The Interrupt class provides a single function wait which is an asyncio coroutine that returns when the interrupt is signalled. 

Coroutines are stateful functions whose execution can be paused. This means that they can yield execution, while they wait on some task or event to complete, but while suspended, they maintain their state.

In [17]:
async def wait_for_timer(cycles): 
    timer.register_map.TLR0 = cycles     # Load the value to count from in the TLR0 register
    timer.register_map.TCSR0.LOAD0 = 1   # Set then clear the LOAD0 bit to trigger the load
    timer.register_map.TCSR0.LOAD0 = 0   
    timer.register_map.TCSR0.ENIT0 = 1   # Set the ENIT0 bit to enable the interrupt output
    timer.register_map.TCSR0.ENT0 = 1    # Set the UDT0 bit to get the timer to count down
    timer.register_map.TCSR0.UDT0 = 1    # Set the ENT0 bit start the timer
    print("starting")
    #The await keyword determines the point where the coroutine yields control and from which execution will resume.
    await interrupt.wait()                
    print("finished")
    timer.register_map.TCSR0.T0INT = 1   # Clear the interrupt after it has been signaled

 asyncio library uses event loops to execute coroutines, in order to executes all ready tasks, polling the status of suspended tasks and scheduling the outstanding ones.

In [18]:
loop = asyncio.get_event_loop()

Coroutines don't execute directly, they are wrapped in tasks

In [21]:
task = loop.create_task(wait_for_timer(1000000000))
loop.run_until_complete(task)

starting
finished
